# Travelling salesman problem - the Nearest Neighbour algorithm

**Set of cities:** cities are points with coordinates x, y on a plane with height as z coordinate. 

**Moving:** cost of going from city A to city B is equal to the Euclidean distance between two cities, if there exists a road. Two scenarios criteria:
* There are all the direct connections / e.g. 80% of possible connections
* The problem is symmetrical / asymmetrical (if asymmetrical – going up is height +10%, going down: -10%)

**Choosing coordinates:** randomly from the range <-100, 100> for x, y and <0, 50> for z.


In [ ]:
import random
from collections import deque
import pandas as pd
import time

In [ ]:
#creating a list of cities
random.seed(12345)
cities = []
for i in range (0,10):
  cities.append([i, random.uniform(-100,100), random.uniform(-100,100), random.uniform(0,50)])

#function measuring the distance from point 1 to point 2
def euclidean(point1, point2, sym):

  euclidean = ((point1[1]-point2[1])**2 + (point1[2]-point2[2])**2 + (point1[3]-point2[3])**2)**(1/2)

  if sym == True:
    return euclidean
  elif sym == False:
    if point2[2]>point1[2]:
      return euclidean*1.1
    elif point2[2]<point1[2]:
      return euclidean*0.9
    else:
      return euclidean

#function creating the directed graph, connecting nodes with each other with weighted edges (using euclidean function)
def create_graph(nodes, conn, sym):

  poss_conn_num = len(nodes) * (len(nodes) - 1)   #number of all possible connections

  if conn > 1 or conn <= 0: raise ValueError

  edges = []

  for k in range (0, len(nodes)):
    for i in range (0, len(nodes)):
      if i != k:
        edges.append([nodes[k], nodes[i], euclidean(nodes[k], nodes[i], sym)])

  if conn <= 1 and conn > 0: 
    rem_conn_num = round(poss_conn_num*(1-conn))     #number of paths we need to remove
    for i in range (0, rem_conn_num):
      del edges[random.randrange(0, len(edges))]
  
  return edges

#creating a graph
graph = create_graph(cities, 0.9, False)

**Task:** Implement a greedy algorithm with nearest neighbour as criterium and another greedy algorithm of your choice to solve the travelling salesman problem.

In [ ]:
citystart = cities[0][0]

#### **Greedy algorithm with the nearest neighbour criterion**

Greedy algorithm - an algorithm that, in order to determine a solution at each step, makes a greedy, that is the most promising at a given moment, selection of a partial solution. In other words, the greedy algorithm does not assess whether it makes sense to perform a given action in the next steps, it makes a locally optimal decision, it makes a choice that seems to be the best at the moment, continuing to solve the sub-problem resulting from the decision made. 

In [ ]:
def greedy_nn(departure_point):
  start = departure_point
  nn_path = [[start], 0]
  no_way_flag = False
  
  while len(nn_path[0])<len(cities):
    visited = set(nn_path[0])
    nndist = [[], 100000]
    for path in graph:
      if path[0][0] == start and path[1][0] not in visited:
        if path[2] < nndist[1]: 
          nndist = [path[1][0], path[2]]
    nn_path[0].append(nndist[0])
    nn_path[1] = nn_path[1] + nndist[1]
    start = nndist[0]
    #print(nn_path[0])
    if nn_path[0][-1] == []:
      no_way_flag = True        #there is no further connection and we cannot create a full cycle (because still not every city is on our list)
      break

  if no_way_flag == False:
    last_point_flag = False     #we did not find a connection to the departure_point yet
    for path in graph:
      if path[0][0] == nn_path[0][-1] and path[1][0] == departure_point:
        nn_path[0].append(departure_point)
        nn_path[1] = nn_path[1] + path[2]
        last_point_flag = True
        break
    if last_point_flag == False:
      nn_path = [None, 0]         #the cycle could not be closed (last point)
  else:
    nn_path = [None, 0]   #there are not all the existing cities on our list
    
  return nn_path

greedy_nn(citystart)

[[0, 6, 2, 9, 4, 3, 1, 7, 5, 8, 0], 716.497298188779]

#### **Greedy algorithm #2 - the nearest neighbour algorithm modification**

We choose a city with the second smallest distance (if it exists, in other case we choose the closest one).

In [ ]:
def greedy_nn_modified(departure_point):
  start = departure_point
  nn_path = [[start], 0]
  no_way_flag = False
  
  while len(nn_path[0])<len(cities):
    visited = set(nn_path[0])
    nndist = [[], 100000]
    nndistold = [[], 100000]

    for path in graph:
      if path[0][0] == start and path[1][0] not in visited:
        if path[2] < nndist[1]: 
          nndistold = nndist
          nndist = [path[1][0], path[2]]
          #print(nndist)

    if nndistold[0] == []:                  #there was just one node to choose from
      nn_path[0].append(nndist[0])
      nn_path[1] = nn_path[1] + nndist[1]
      #print(nn_path[0])
      start = nndist[0]
    else:                                 #we found the second closest node
      nn_path[0].append(nndistold[0])
      nn_path[1] = nn_path[1] + nndistold[1]
      #print(nn_path[0])
      start = nndistold[0]

    if nn_path[0][-1] == []:
      no_way_flag = True        #there is no further connection and we cannot create a full cycle (because still not every city is on our list)
      break

  if no_way_flag == False:
    last_point_flag = False     #we did not find a connection to the departure_point yet
    for path in graph:
      if path[0][0] == nn_path[0][-1] and path[1][0] == departure_point:
        nn_path[0].append(departure_point)
        nn_path[1] = nn_path[1] + path[2]
        last_point_flag = True
        break
    if last_point_flag == False:
      nn_path = [None, 0]         #the cycle could not be closed (last point)
  else:
    nn_path = [None, 0]   #there are not all the existing cities on our list
    
  return nn_path

citystart = cities[0][0]
greedy_nn_modified(citystart)

[[0, 2, 4, 3, 1, 6, 7, 5, 8, 9, 0], 838.1440259479851]

#### **Technical note**
To test/show the effectiveness and co-replenishment of the algorithms, we can use:
* seed(1234), range(0,10), 90%

the NN algorithm enters the while loop, but adds an empty list (no_way_flag -> True), the modified algorithm finds the complete cycle 
* seed(123), range(0,10), 90% 

none of the algorithms find the last path back to the starting point (last_point_flag -> False) 
* seed(123), range(0,10), 100% 

both algorithms find the best path, but the modified algorithm's one is much longer

* seed(12345), range(0,10), 90%

the NN algorithm finds the best path, while the modified algorithm enters the while loop, but adds an empty list (no_way_flag -> True)